Using BERT and Tensorflow 2.0,write code to classify emails as spam or not spam. BERT will be used to generate sentence encoding for all emails and after that use a simple neural network with one drop out layer and one output layer. 


In [1]:
!pip install tensorflow_text


     |████████████████████████████████| 4.9 MB 5.3 MB/s 


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd


In [3]:
df = pd.read_csv("spam.csv")
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
# basic analysis
df.groupby('Category').describe()

Message                                                               
           count unique                                                top freq
Category                                                                       
ham         4825   4516                             Sorry, I'll call later   30
spam         747    641  Please call our customer service representativ...    4

In [5]:
# create spam column
df['spam'] = df['Category'].apply(lambda x: 1 if x == 'spam' else 0)
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [6]:
# train test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Message'], df['spam'], test_size= 0.2, stratify= df['spam'])

In [7]:
y_train.value_counts()

0    3859
1     598
Name: spam, dtype: int64

In [8]:
y_test.value_counts()

0    966
1    149
Name: spam, dtype: int64

In [9]:
# embedding using BERT

bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [10]:
# define simple function that takes a simple sentence a gives an embedded vector
def get_sentence_embedding(sentences):
  preprocessed_text = bert_preprocess(sentences)

  return bert_encoder(preprocessed_text)['pooled_output']


In [11]:
get_sentence_embedding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"
                        
])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8435166 , -0.5132724 , -0.88845706, ..., -0.7474883 ,
        -0.7531471 ,  0.91964483],
       [-0.8720836 , -0.50544   , -0.9444667 , ..., -0.8584748 ,
        -0.71745366,  0.88082993]], dtype=float32)>

In [12]:
e = get_sentence_embedding([
    "banana", 
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
                        
])

In [13]:
# use cosine similarity to compare two vectors
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([e[0]], [e[1]])

array([[0.99110895]], dtype=float32)

In [14]:
cosine_similarity([e[0]], [e[3]])

array([[0.8470383]], dtype=float32)

In [15]:
cosine_similarity([e[3]], [e[4]])

array([[0.9872036]], dtype=float32)

types of models
1. sequenttial
2. functional

 https://becominghuman.ai/sequential-vs-functional-model-in-keras-20684f766057

In [19]:
# build a functional moddel
text_input = tf.keras.layers.Input(shape= (), dtype= tf.string, name= 'text')

preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# neural netwrok layers
l = tf.keras.layers.Dropout(0.1, name= "dropout")(outputs['pooled_output'])

# Dense layer
l = tf.keras.layers.Dense(1, activation= "sigmoid", name= "output")(l)  # in functional model, pass previous layer

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs= [text_input], outputs= [l])
model.summary()


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [20]:
model.compile(optimizer= 'adam',
              loss= 'binary_crossentropy',
              metrics= ['accuracy'])

Train the model

In [21]:
model.fit(X_train, y_train, epochs= 5)

Epoch 1/5
140/140 [==============================] - 105s 639ms/step - loss: 0.3430 - accuracy: 0.8631
Epoch 2/5
140/140 [==============================] - 85s 607ms/step - loss: 0.2401 - accuracy: 0.8934
Epoch 3/5
140/140 [==============================] - 83s 595ms/step - loss: 0.2005 - accuracy: 0.9188
Epoch 4/5
140/140 [==============================] - 83s 595ms/step - loss: 0.1719 - accuracy: 0.9392
Epoch 5/5
140/140 [==============================] - 83s 593ms/step - loss: 0.1556 - accuracy: 0.9482


In [22]:
model.evaluate(X_test, y_test)

35/35 [==============================] - 22s 590ms/step - loss: 0.1490 - accuracy: 0.9399


[0.1489858627319336, 0.9399102926254272]

Inference


In [23]:
emails = [
    'Reply to win Â£100 weekly! Where will the 2006 FIFA World Cup be held? Send STOP to 87239 to end service',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]

model.predict(emails)

array([[0.49222416],
       [0.58171713],
       [0.400995  ],
       [0.04183003],
       [0.01719701]], dtype=float32)

values > .5 spam

exercise: dataset text classification with bert tesorflow